# 머신러닝 데이터 전처리
> 수집한 데이터를 머신러닝 알고리즘을 운용할 수 있는 형태로 가공하는 작업을 지칭합니다.  
결측치, 범주형 데이터, 문자열, 날짜 형식 데이터 등 일반적인 데이터에서 발생 가능한 문제를 해결하는 일반적인 방법론을 지칭 하기도 합니다.  
다만 데이터의 특성 및 사용해야 하는 모델 케이스에 따라 수행해야하는 전처리 과정이 달라 기술적인 어려움이 있습니다.  
이번 시간에는 기본적인 전처리 방법들을 소개하고 샘플데이터에 적용하며 전처리 방법을 익혀 보겠습니다.  

## titanic 데이터 설명
> 타이타닉호에 승선했던 탑승자 정보를 담고 있는 데이터입니다. 데이터분석 필드에서는 교육용 및 입문용 데이터로 오랫동안 사랑받아 온 데이터이며  
탑승자 정보를 바탕으로 이 탑승자가 생존했는지 혹은 사망했는지 분류하는 모델링을 목적으로 제작 된 데이터입니다.  

**columns 정보**
- pclass : 객실 등급  
- survived : 생존 유무
- name : 칭호를 포함한 이름
- sex : 성별
- age : 나이
- sibsp : 형제 혹은 부부의 수
- parch : 부모, 혹은 자녀의 수
- ticket : 탑승권 종류 (각 숫자가 의미하는 바는 데이터를 통해 알 수 있는 것이 없을 수 있습니다.)
- fare : 지불한 운임
- cabin : 객실정보
- embarked : 선착장정보
- boat : 탈출한 보트가 있다면 boat 번호
- body : 사망자의 시신 수습 후 부여한 일련번호
- home : 출신

## 결측 데이터 처리

In [ ]:
# 필요모듈 import


In [ ]:
# 타이타닉 데이터 로드


In [ ]:
# 결측 데이터 분포 시각화


In [ ]:
# 상관관계분석


이 중 cabin, home.dest 컬럼 데이터의 경우 결측치 비중이 높고 주변 데이터로부터 빈 데이터를 유추할 수 있는 힌트가 없어 삭제 처리 합니다.

In [ ]:
# 결측치 비율 계산


In [ ]:
# 여러가지 방법으로 컬럼 삭제


In [ ]:
# 결측치 처리


boat, body의 경우 유추하여 값을 채워 넣을 수 있는 데이터는 없지만 데이터의 존재 자체로서 생존 여부를 판단 할 수 있는 데이터이기에 데이터를 명목화 시키겠습니다.

## 범주형 변수의 처리
> 카테고리컬 데이터라고도 하며 데이터가 특정 구간에 속하여 구분이 가능한 데이터 분포를 보이는 변수의 경우 범주형 데이터라 합니다.  
범주형 데이터의 경우 보통 문자열 구분이 되어있어 컴퓨터가 인식 가능한 숫자형태로 바꿔주는데 이를 더미화, 혹은 원핫인코딩이라 합니다.

In [ ]:
# 보트 컬럼의 결측치 여부를 이진형 데이터로 치환


In [ ]:
# True, False 변환


In [ ]:
# 컬럼데이터가 True면 1, False면 0 으로 변환하는 커스텀함수 제작


In [ ]:
# 컬럼데이터에 위에서 제작한 커스텀함수 적용


## 카테고리가 여러개인 변수 처리 (문자열 처리)

이름의 경우 미들네임에 따라 생존에 영향을 줄 수 있는 경우가 있습니다. 전체 미들네임에 해당하는 카테고리를 선택하지 않고 일부만을 데이터화 시킵니다.

In [ ]:
# name 컬럼 첫번째 인덱스 데이터 선택해서 test_text로 저장


In [ ]:
# 팬시인덱싱 내가 설정한 조건에 따라 데이터 선별


In [ ]:
# 선별텍스트에서 문자열만 선택


In [ ]:
# 미들네임 선택 함수


In [ ]:
# 미들네임 선택 함수 적용 후 name 컬럼 원본값 변경


In [ ]:
# name 컬럼의 문자열 그대로는 학습에 사용이 불가능하기에 이를 다시 숫자형태로 치환 + 기타 미들네임을 0으로 변환


In [ ]:
# 커스텀함수 적용


In [ ]:
# 데이터프레임 내부에 카테고리컬 컬럼을 원핫인코딩 변환 작업


## 타겟, 학습데이터 분할

In [ ]:
# 타겟데이터 분할

# 학습데이터 분할


## 모델학습

In [ ]:
# 모델 생성


In [ ]:
# 모델 학습


In [ ]:
# 모델 예측값 저장


In [ ]:
# 모델 평가


## 날짜형식 데이터 전처리

### 사용 데이터 간략 설명
> 한국에너지관리공단에서 제공한 전력사용량 데이터  
1시간 간격으로 수집 된 60개 건물들의 2020년 6월 1일 부터 2020년 8월 24일까지의 데이터  
건물정보와 기후정보를 활용한 전력사용량을 예측하기 위한 데이터셋  
대회홈페이지 : https://dacon.io/competitions/official/235736/overview/description
>> 각 변수(컬럼) 설명  
>>- num : 건물번호  
>>- date_time : 데이터가 수집 된 날짜, 시간  
>>- 전력사용량 : 수집 된 시점에 사용한 전력량  
>>- 비전기냉방설비운영 : 0-미운영, 1-운영  
>>- 태양광보유 : 0-미보유, 1-보유

In [ ]:
# 데이터 로딩


In [ ]:
# date_time컬럼 날짜 형식으로 변환 후 새로운 변수로 저장


In [ ]:
# datetime 형식의 데이터에서 시간 속성 값 추출 테스트


In [ ]:
# 함수 제작 및 적용
